In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import fiona
import glob
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from functools import partial
import pyproj

from methods import read_shst_extract

In [47]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
data_interim_dir = "../../data/interim/"

In [3]:
# load base network links and shapes from step3 (before third-party conflation)

link_file = data_interim_dir + "step3_join_shst_extraction_with_osm/" + "link.json"
with open(link_file) as f:
    link_json = json.load(f)
link_df = pd.DataFrame(link_json)

shape_gdf = gpd.read_file(data_interim_dir + "step3_join_shst_extraction_with_osm/" 
                          + "shape.geojson")

link_df = pd.merge(link_df,
                   shape_gdf[["id", "geometry"]],
                   how = "left",
                   left_on = "shstGeometryId",
                   right_on = "id")

link_df = gpd.GeoDataFrame(link_df, geometry = link_df["geometry"],
                            crs={"init" : "epsg:4326"})

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [5]:
# load conflation result from step4c and merge columns 'tomtom_shieldnum', 'tomtom_rtedir' to the base network

conflation_df = pd.read_csv("../../data/interim/conflation_result.csv")
print(conflation_df.columns)

link_df = pd.merge(link_df,
                   conflation_df[["shstReferenceId", 'tomtom_shieldnum', 'tomtom_rtedir']],
                   how = "left",
                   on = 'shstReferenceId')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
# Load pems raw data and convert to gdf

pems_file = "../../data/external/mtc/pems_period.csv"
pems_df = pd.read_csv(pems_file)

pems_df["geometry"] = [Point(xy) for xy in zip(pems_df.longitude, pems_df.latitude)]

pems_gdf = gpd.GeoDataFrame(pems_df, geometry = pems_df["geometry"],
                            crs={"init" : "epsg:4326"})

pems_gdf = pems_gdf[~((pems_gdf.longitude.isnull()) | (pems_gdf.latitude.isnull()))]

# Nearest match based on shieldnum and direction

### Prepare for nearest match

In [9]:
# convert crs to meter based

pems_gdf = pems_gdf.to_crs(epsg = 26915)
link_df = link_df.to_crs(epsg = 26915)

In [10]:
pems_gdf.shape

(76057, 23)

In [11]:
pems_gdf.route.value_counts().sum()

76057

In [12]:
link_df.sindex

rtree.index.Index(bounds=[-2175501.4170011017, 4474858.796580055, -2012056.1252066875, 4771895.207762718], size=1705772)

In [15]:
pems_gdf.columns

Index(['station', 'district', 'route', 'direction', 'type', 'time_period',
       'lanes', 'median_flow', 'avg_flow', 'sd_flow', 'median_speed',
       'avg_speed', 'sd_speed', 'median_occup', 'avg_occup', 'sd_occupancy',
       'days_observed', 'state_pm', 'abs_pm', 'latitude', 'longitude', 'year',
       'geometry'],
      dtype='object')

In [16]:
link_df.tomtom_shieldnum.unique()

array([' ', nan, '85', '580', '24', '12', '4', '82', 'G4', '380', '116',
       '280', '185', '80', '121', '9', '1', '114', '880', '123', 'G2',
       '29', '128', '13', 'G8', '84', '680', '101', '92', '238', '10',
       '780', '131', 'J2', '113', '35', '37', '130', '109', '17', '205',
       '112', '220', '262', '237', '61', '2063', '152', '87', '505',
       '260', '160', '242', 'G3', '221', 'G9', 'G7', '93', '77', '25',
       '980', '156'], dtype=object)

In [17]:
pems_gdf.route.unique()

array([101,  80, 680, 280, 880, 580,  24,   4, 238,  85,  87,  17, 237,
       242,  92,   1, 980,  25,  37, 948, 780,  84,  29, 156, 205, 380,
        12, 152, 160], dtype=int64)

### Write out links whose shieldnum appear in the pems data
There are condidate links for PEMs conflation

In [18]:
# routes not in tomtom_shieldnum
[c for c in pems_gdf.route.unique().astype(str) if c not in link_df.tomtom_shieldnum.unique()]

['948']

In [19]:
# links included in pems routes 
interest_facility_df = link_df[link_df.tomtom_shieldnum.isin(pems_gdf.route.unique().astype(str))]

In [20]:
interest_facility_df = interest_facility_df.to_crs(epsg = 4326)

In [21]:
# export for QAQC
interest_facility_df[
    ["roadway", "shstReferenceId", "geometry"]].to_file(data_interim_dir + "mtc/link_candidates_for_pems.geojson",
                            driver = "GeoJSON")

In [22]:
# check unique 'tomtom_shieldnum' + 'tomtom_rtedir' comb in the links data
interest_facility_df.groupby(["tomtom_shieldnum", "tomtom_rtedir"]).count()

shstReferenceId  id_x  shstGeometryId  \
tomtom_shieldnum tomtom_rtedir                                          
1                                          1205  1205            1205   
                 N                          121   121             121   
                 S                          140   140             140   
101                                         205   205             205   
                 N                          284   284             284   
...                                         ...   ...             ...   
92                                          180   180             180   
                 E                           29    29              29   
                 W                           26    26              26   
980              E                            6     6               6   
                 W                            8     8               8   

                                fromIntersectionId  toIntersectionId     u  \
tomtom_shieldnum tomtom_rtedir                                               
1                                             1205              1205  1205   
                 N                             121               121   121   
                 S                             140               140   140   
101                                            205               205   205   
                 N                             284               284   284   
...                                            ...               ...   ...   
92                                             180               180   180   
                 E                              29                29    29   
                 W                              26                26    26   
980              E                               6                 6     6   
                 W                               8                 8     8   

                                   v  link  oneWay  roundabout  ...   ref  \
tomtom_shieldnum tomtom_rtedir                                  ...         
1                               1205  1205    1205        1205  ...  1205   
                 N               121   121     121         121  ...   121   
                 S               140   140     140         140  ...   140   
101                              205   205     205         205  ...   205   
                 N               284   284     284         284  ...   284   
...                              ...   ...     ...         ...  ...   ...   
92                               180   180     180         180  ...   180   
                 E                29    29      29          29  ...    29   
                 W                26    26      26          26  ...    26   
980              E                 6     6       6           6  ...     6   
                 W                 8     8       8           8  ...     8   

                                service  tunnel  width  roadway  drive_access  \
tomtom_shieldnum tomtom_rtedir                                                  
1                                  1205    1205   1205     1205          1205   
                 N                  121     121    121      121           121   
                 S                  140     140    140      140           140   
101                                 205     205    205      205           205   
                 N                  284     284    284      284           284   
...                                 ...     ...    ...      ...           ...   
92                                  180     180    180      180           180   
                 E                   29      29     29       29            29   
                 W                   26      26     26       26            26   
980              E                    6       6      6        6             6   
                 W                    8       8      8        8             8   

           

In [23]:
# check unique 'route' + 'direction' comb in the pems data
pems_gdf.groupby(["route", "direction"]).count()

station  district  type  time_period  lanes  median_flow  \
route direction                                                             
1     N               35        35    35           35     35           35   
      S               30        30    30           30     30           30   
4     E             1756      1756  1756         1756   1756         1756   
      W             1800      1800  1800         1800   1800         1800   
12    E              110       110   110          110    110          110   
      W              100       100   100          100    100          100   
17    N              700       700   700          700    700          700   
      S              439       439   439          439    439          439   
24    E             1221      1221  1221         1221   1221         1221   
      W             1455      1455  1455         1455   1455         1455   
25    N              182       182   182          182    182          182   
      S              183       183   183          183    183          183   
29    N               65        65    65           65     65           65   
      S               75        75    75           75     75           75   
37    E              283       283   283          283    283          283   
      W              352       352   352          352    352          352   
80    E             5167      5167  5167         5167   5167         5167   
      W             5089      5089  5089         5089   5089         5089   
84    E              355       355   355          355    355          355   
      W              330       330   330          330    330          330   
85    N             1670      1670  1670         1670   1670         1670   
      S             1635      1635  1635         1635   1635         1635   
87    N              652       652   652          652    652          652   
      S              674       674   674          674    674          674   
92    E              727       727   727          727    727          727   
      W              729       729   729          729    729          729   
101   N             9914      9914  9914         9914   9914         9914   
      S             9344      9344  9344         9344   9344         9344   
152   E               30        30    30           30     30           30   
      W               25        25    25           25     25           25   
156   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
160   N               15        15    15           15     15           15   
      S               15        15    15           15     15           15   
205   E               15        15    15           15     15           15   
      W               19        19    19           19     19           19   
237   E              388       388   388          388    388          388   
      W              509       509   509          509    509          509   
238   N              291       291   291          291    291          291   
      S              299       299   299          299    299          299   
242   N              224       224   224          224    224          224   
      S              232       232   232          232    232          232   
280   N             1832      1832  1832         1832   1832         1832   
      S             1794      1794  1794         1794   1794         1794   
380   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
580   E             2765      2765  2765         2765   2765         2765   
      W             2671      2671  2671         2671   2671         2671   
680   N             4358      4358  4358         4358   4358         4358   
      S             4687      4687  4687         4687   4687         4687   
780   E      

### Match Pems to the nearest link that has same shieldnum and direction

In [24]:
pems_gdf["type"].value_counts()

ML    60319
OR     9309
FR     5733
FF      666
HV       30
Name: type, dtype: int64

In [25]:
# dictionary for pems type and roadway type
roadway_type_list = link_df.roadway.unique()

pems_type_list = pems_gdf["type"].unique()

pems_roadway_crosswalk = {"ML" : ['tertiary', 'primary', 'secondary', 'motorway', 'trunk'],
                          "HV" : [c for c in roadway_type_list if c.endswith("_link")],
                          "FF" : [c for c in roadway_type_list if c.endswith("_link")],
                          "OR" : [c for c in roadway_type_list if c.endswith("_link")],
                          "FR" : [c for c in roadway_type_list if c.endswith("_link")]
                         }

In [26]:
pems_roadway_crosswalk

{'ML': ['tertiary', 'primary', 'secondary', 'motorway', 'trunk'],
 'HV': ['primary_link',
  'motorway_link',
  'trunk_link',
  'secondary_link',
  'tertiary_link'],
 'FF': ['primary_link',
  'motorway_link',
  'trunk_link',
  'secondary_link',
  'tertiary_link'],
 'OR': ['primary_link',
  'motorway_link',
  'trunk_link',
  'secondary_link',
  'tertiary_link'],
 'FR': ['primary_link',
  'motorway_link',
  'trunk_link',
  'secondary_link',
  'tertiary_link']}

In [27]:
%%time
# match based on pems route, direction, and tomtom shieldnum, rtedir

offset = 100

pems_route_list = pems_gdf.route.unique().tolist()

pems_match_gdf = gpd.GeoDataFrame()

for route in pems_route_list:
    
    print("pems route id {}".format(route))
    
    pems_subset_gdf = pems_gdf[(pems_gdf.route == route)].copy()
    
    dir_list = pems_subset_gdf.direction.unique().tolist()
    
    for direction in dir_list:
        
        print("\t pems direction {}".format(direction))
        
        pems_subset_gdf = pems_gdf[(pems_gdf.route == route) & 
                                   (pems_gdf.direction == direction)].copy()
        
        type_list = pems_subset_gdf["type"].unique().tolist()
        
        for ptype in type_list:
            
            print("\t\t pems type {}".format(ptype))
            
            pems_subset_gdf = pems_gdf[(pems_gdf.route == route) &
                                       (pems_gdf.direction == direction) &
                                       (pems_gdf["type"] == ptype)].copy()
            
            bbox = pems_subset_gdf.bounds + [-offset, -offset, offset, offset]
    
            line = link_df[(link_df.tomtom_shieldnum == str(route)) & 
                           (link_df.tomtom_rtedir == direction) &
                           (link_df.roadway.isin(pems_roadway_crosswalk[ptype])
                           )].copy()
            
            if len(line) == 0:
                print("\t\t there is no link with tomtom label direction {}, route {}, roadway {}, matching to the closest\
                      {}".format(
                    direction, route, ptype, pems_roadway_crosswalk[ptype]
                ))
                
                line = link_df[(link_df.roadway.isin(pems_roadway_crosswalk[ptype]))
                              ].copy()
        
            hits = bbox.apply(lambda row: list(line.sindex.intersection(row)),
                  axis = 1)
            
            tmp = pd.DataFrame({
                # index of points table
                "pt_idx": np.repeat(hits.index, hits.apply(len)),
                # ordinal position of line - access via iloc later
                "line_i": np.concatenate(hits.values)
            })
        
            # join with pems
        
            tmp.set_index(["pt_idx"], inplace = True)
        
            tmp = tmp.join(pems_subset_gdf[["station", "longitude", "latitude", "route", "direction", "type","geometry"]
                                          ].rename(
                                columns = {"geometry" : "point"}), 
                           how = "left")
            
            # join with links
        
            tmp.set_index(["line_i"], inplace = True)
        
            tmp = tmp.join(line[["shstReferenceId", "roadway","tomtom_shieldnum", "tomtom_rtedir", "geometry"]
                           ].reset_index(drop=True), 
                       how="left")
        
            # find closest line to point
        
            tmp = gpd.GeoDataFrame(tmp, geometry = tmp["geometry"], crs = pems_gdf.crs)
        
            tmp["snap_distance"]  = tmp.geometry.distance(gpd.GeoSeries(tmp.point))
        
            tmp.sort_values(by = ["snap_distance"], inplace = True)
        
            closest = tmp.groupby(["station", "longitude", "latitude"]).first().reset_index()
        
            pems_match_gdf = pd.concat([pems_match_gdf, closest],
                                       sort = False,
                                       ignore_index = True)

pems route id 101
	 pems direction S
		 pems type ML
		 pems type FR
		 pems type OR
		 pems type FF
	 pems direction N
		 pems type ML
		 pems type FR
		 pems type OR
		 pems type FF
pems route id 80
	 pems direction W
		 pems type ML
		 pems type FR
		 pems type FF
		 pems type OR
	 pems direction E
		 pems type ML
		 pems type FF
		 pems type OR
		 pems type FR
pems route id 680
	 pems direction N
		 pems type ML
		 pems type OR
		 there is no link with tomtom label direction N, route 680, roadway OR, matching to the closest                      ['primary_link', 'motorway_link', 'trunk_link', 'secondary_link', 'tertiary_link']
		 pems type FR
		 there is no link with tomtom label direction N, route 680, roadway FR, matching to the closest                      ['primary_link', 'motorway_link', 'trunk_link', 'secondary_link', 'tertiary_link']
		 pems type FF
		 there is no link with tomtom label direction N, route 680, roadway FF, matching to the closest                      ['primary

pems route id 780
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 84
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 29
	 pems direction N
		 pems type ML
		 pems type FR
		 pems type OR
	 pems direction S
		 pems type ML
		 pems type FR
		 pems type OR
pems route id 156
	 pems direction W
		 pems type ML
		 there is no link with tomtom label direction W, route 156, roadway ML, matching to the closest                      ['tertiary', 'primary', 'secondary', 'motorway', 'trunk']
	 pems direction E
		 pems type ML
		 there is no link with tomtom label direction E, route 156, roadway ML, matching to the closest                      ['tertiary', 'primary', 'secondary', 'motorway', 'trunk']
pems route id 205
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 380
	 pems direction E
		 pems type ML
	 pems direction W
		 pems type ML
pems route id 12
	 pems direction E
		 pems type ML
	 pe

In [28]:
pems_match_gdf

,station,longitude,latitude,route,direction,type,point,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,geometry,snap_distance
0,400000,-122.547963,38.081498,101,S,ML,POINT (-2115334.471 4652792.447),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,"LINESTRING (-2115401.041 4652959.381, -2115140...",2.867350
1,400000,-122.547606,38.081167,101,S,ML,POINT (-2115315.560 4652743.599),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,"LINESTRING (-2115401.041 4652959.381, -2115140...",3.174880
2,400002,-122.328465,37.584097,101,S,ML,POINT (-2115131.411 4589330.976),dfd26b2508d008ad867b43ce2eae005f,motorway,101,S,"LINESTRING (-2117606.567 4590672.775, -2117515...",3.106433
3,400043,-122.403092,37.753591,101,S,ML,POINT (-2115244.799 4610955.232),6d8b2509ddc5a1a71b55ff5b569e136a,motorway,101,S,"LINESTRING (-2114949.615 4612440.258, -2114990...",2.475063
4,400069,-121.977493,37.385758,101,S,ML,POINT (-2090942.692 4555908.530),7dc48aa303c48fb6eea805245d5af370,motorway,101,S,"LINESTRING (-2091431.901 4556300.267, -2090941...",4.480192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,406891,-121.381462,36.976267,152,W,ML,POINT (-2052013.684 4491230.905),1699819416ecfa0f388a955212d4f828,trunk,152,,"LINESTRING (-2051906.805 4491544.919, -2051911...",1.213285
2853,415729,-121.751557,38.016167,160,S,FF,POINT (-2046319.733 4620748.189),db201e425e9d0e62a6e581682a235332,motorway_link,,,"LINESTRING (-2046348.712 4620682.244, -2046389...",72.031723
2854,415730,-121.751549,38.016323,160,S,FF,POINT (-2046313.021 4620765.669),db201e425e9d0e62a6e581682a235332,motorway_link,,,"LINESTRING (-2046348.712 4620682.244, -2046389...",90.739225
2855,415727,-121.751643,38.013534,160,N,FF,POINT (-2046428.610 4620451.669),56f05bac9e4f37348ac809be98de2be3,motorway_link,,,"LINESTRING (-2046454.028 4620354.160, -2046431...",11.871443


In [29]:
pems_nearest_gdf = pd.merge(pems_gdf,
                            pems_match_gdf.drop(["point", "geometry"], axis = 1),
                            how = "left",
                            on = ["station", "longitude", "latitude", "route", "direction", "type"])

In [32]:
pems_nearest_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 76057 entries, 0 to 76056
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   station           76057 non-null  int64   
 1   district          76057 non-null  int64   
 2   route             76057 non-null  int64   
 3   direction         76057 non-null  object  
 4   type              76057 non-null  object  
 5   time_period       76057 non-null  object  
 6   lanes             76057 non-null  int64   
 7   median_flow       76057 non-null  float64 
 8   avg_flow          76057 non-null  float64 
 9   sd_flow           76057 non-null  float64 
 10  median_speed      60350 non-null  float64 
 11  avg_speed         60350 non-null  float64 
 12  sd_speed          60350 non-null  float64 
 13  median_occup      76057 non-null  float64 
 14  avg_occup         76057 non-null  float64 
 15  sd_occupancy      76057 non-null  float64 
 16  days_observed 

In [34]:
# match results
print('{} pems stations matched'.format(pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()].station.nunique()))

print('{} pems stations not matched'.format(pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.isnull()].station.nunique()))

2440 pems stations matched
1286 pems stations not matched


In [35]:
# road type breakdown of the matched station data points
pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()]["type"].value_counts()

ML    54576
OR     1476
FR      658
FF      201
HV       15
Name: type, dtype: int64

# For unmatched by shieldnum+direction, do FT match and ShSt match

### FT (facility type) match

In [43]:
# for PEMS failed to match by sheildnum+direction nearest method
pems_nearest_not_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.isnull()].copy()

# drop duplicates
unique_pems_nearest_not_matched_df = pems_nearest_not_matched_df.drop_duplicates(
    subset = ["station", "type", "latitude", "longitude"]
)

print('{} PEMS records did not find a nearest sheildnum+direction match, \
representing {} unique PEMS station/type/location'.format(pems_nearest_not_matched_df.shape[0],
                                                                  unique_pems_nearest_not_matched_df.shape[0]))

19131 PEMS records, did not find a nearest sheildnum+direction match, representing 1408 unique PEMS station/type/location


In [44]:
# set link_df to EPSG 4326
link_df = link_df.to_crs({'init': 'epsg:4326'})

In [45]:
# set key parameters and functions

proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
offset = 100

def geodesic_point_buffer(lat, lon, meters):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(meters)  # distance in metres
    return Polygon(transform(project, buf).exterior.coords[:])

def links_within_stop_buffer(drive_link_df, stop, buffer_radius = 25):
    """
    find the links that are within buffer of nodes
    """
    
    stop_buffer_df = stop.copy()
    stop_buffer_df["geometry"] = stop_buffer_df.apply(lambda x: geodesic_point_buffer(x.latitude, x.longitude, buffer_radius), 
                                                      axis = 1)
    
    stop_buffer_df = gpd.GeoDataFrame(stop_buffer_df, geometry = stop_buffer_df["geometry"], crs = {'init' : 'epsg:4326'})
    
    stop_buffer_link_df = gpd.sjoin(drive_link_df, stop_buffer_df[["geometry", "type"]], 
                                             how = "left", op = "intersects")
    
    stop_buffer_link_df = stop_buffer_link_df[stop_buffer_link_df["type"].notnull()]
    
    return stop_buffer_link_df["shstReferenceId"].tolist()

In [46]:
%%time

# match based on facility type

pems_matched_ft_gdf = gpd.GeoDataFrame()

for i in range(len(unique_pems_nearest_not_matched_df)):
    
    row_df = unique_pems_nearest_not_matched_df.iloc[[i]][["station", "type", "latitude", "longitude"]].copy()
    
    print("process station {}, type {}".format(row_df["station"].iloc[0], row_df["type"].iloc[0]))
    
    link = link_df[(link_df.roadway.isin(pems_roadway_crosswalk[row_df["type"].iloc[0]]))].copy()
    
    links_within_buffer = links_within_stop_buffer(link, row_df, offset)
    
    row_df["geometry"] = row_df.apply(lambda x: Point(x.longitude, x.latitude), axis = 1)
    
    # find closest line to point
    
    links_within_buffer_gdf = link_df[link_df.shstReferenceId.isin(links_within_buffer)].copy()
    
    links_within_buffer_gdf["station"] = row_df["station"].iloc[0]
    links_within_buffer_gdf["type"] = row_df["type"].iloc[0]
    links_within_buffer_gdf["latitude"] = row_df["latitude"].iloc[0]
    links_within_buffer_gdf["longitude"] = row_df["longitude"].iloc[0]
    links_within_buffer_gdf["point"] = row_df["geometry"].iloc[0]
        
    links_within_buffer_gdf["snap_distance"] = links_within_buffer_gdf.geometry.distance(
        gpd.GeoSeries(links_within_buffer_gdf.point)
    )
        
    links_within_buffer_gdf.sort_values(by = ["snap_distance"], inplace = True)
        
    closest_link_gdf = links_within_buffer_gdf.groupby(["station", "longitude", "latitude"]).first().reset_index()
    
    pems_matched_ft_gdf = pd.concat([pems_matched_ft_gdf, closest_link_gdf], sort = False, ignore_index = True)

process station 400009, type ML


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<timed exec>:27: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

<timed exec>:27: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None



process station 400060, type ML


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<timed exec>:27: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

<timed exec>:27: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None



process station 400108, type ML


...

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<timed exec>:27: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

<timed exec>:27: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None



process station 419663, type OR


C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Wall time: 1h 40min 25s


<timed exec>:27: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

<timed exec>:27: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: None



In [48]:
pems_use_ft_result_df = pd.merge(pems_nearest_not_matched_df.drop("shstReferenceId", axis = 1),
                                 pems_matched_ft_gdf[["shstReferenceId", 'station', 'longitude', 'latitude', 'type']],
                                 how = "left",
                                 on = ['station', 'type','longitude', 'latitude'])

In [49]:
# stats

print('additional {} stations matched'.format(pems_use_ft_result_df.station.nunique()))

print('facility types of still unmatched station data points: \n{}'.format(
    pems_use_ft_result_df[pems_use_ft_result_df.shstReferenceId.isnull()]['type'].value_counts()))


additional 1286 stations matched
facility types of still unmatched station data points: 
OR    236
FR    207
FF     66
ML     50
Name: type, dtype: int64


In [119]:
# stop_buffer_df = row_df.copy()
# stop_buffer_df["geometry"] = stop_buffer_df.apply(lambda x: geodesic_point_buffer(x.latitude, x.longitude, 5), 
#                                                       axis = 1)

# stop_buffer_df = gpd.GeoDataFrame(stop_buffer_df, geometry = stop_buffer_df["geometry"], crs = {'init' : 'epsg:4326'})

# stop_buffer_link_df = gpd.sjoin(link, stop_buffer_df[["geometry", "type"]], 
#                                              how = "left", op = "intersects")

In [122]:
# stop_buffer_link_df[stop_buffer_link_df["type"].notnull()].shstReferenceId

226554    3fe69a55506d7a078c327d075af084c8
Name: shstReferenceId, dtype: object

In [111]:
# stop_buffer_df

,type,latitude,longitude,geometry
15,ML,37.863905,-122.303083,POLYGON ((-122.3030261782874 37.86390499998629...


In [88]:
# unique_pems_nearest_not_matched_df.iloc[[0]]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
15,400009,4,80,W,ML,AM,5,16085.0,15930.111111,577.366524,...,10.94,37.863905,-122.303083,2005,POINT (-2101884.901027649 4620392.963378702),NaN,NaN,NaN,NaN,NaN


### Add shst match

In [51]:
pems_shst_match_df = read_shst_extract(data_interim_dir + "mtc/", "*pems.out.matched.geojson")

pems_shst_match_df.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                     "shstToIntersectionId" : "toIntersectionId",
                                     "pp_station" : "station",
                                     "pp_longitude" : "longitude",
                                     "pp_latitude" : "latitude",
                                     "referenceId" : "shstReferenceId"},
                          inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/mtc\car_rules\pems.out.matched.geojson
----------finished reading shst extraction data-------------


In [53]:
# merge shst match result to the unmatched through sheild_num+direction

pems_nearest_not_matched_df = pems_nearest_not_matched_df.round({"longitude" : 6, "latitude" : 6})

pems_use_shst_result_df = pd.merge(pems_nearest_not_matched_df.drop("shstReferenceId", axis = 1),
                                   pems_shst_match_df[["shstReferenceId", 'station', 'longitude', 'latitude', 'source']],
                                   how = "left",
                                   on = ['station', 'longitude', 'latitude'])

In [54]:
# stats

print('additional {} stations matched'.format(pems_use_shst_result_df.station.nunique()))

print('facility types of still unmatched station data points: \n{}'.format(
    pems_use_shst_result_df[pems_use_shst_result_df.shstReferenceId.isnull()]['type'].value_counts()))

additional 1286 stations matched
facility types of still unmatched station data points: 
Series([], Name: type, dtype: int64)


In [57]:
pems_use_shst_result_df.station.nunique()

1286

# Concat match result from two methods

In [171]:
pems_nearest_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()].copy()

In [59]:
pems_conflation_result_df = pd.concat(
        [pems_nearest_matched_df, pems_use_shst_result_df],
        sort = False,
        ignore_index = True
)

In [172]:
pems_conflation_result_df = pd.concat(
        [pems_nearest_matched_df, pems_use_ft_result_df],
        sort = False,
        ignore_index = True
)

In [173]:
pems_conflation_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 76057 entries, 0 to 76056
Data columns (total 28 columns):
station             76057 non-null int64
district            76057 non-null int64
route               76057 non-null int64
direction           76057 non-null object
type                76057 non-null object
time_period         76057 non-null object
lanes               76057 non-null int64
median_flow         76057 non-null float64
avg_flow            76057 non-null float64
sd_flow             76057 non-null float64
median_speed        60350 non-null float64
avg_speed           60350 non-null float64
sd_speed            60350 non-null float64
median_occup        76057 non-null float64
avg_occup           76057 non-null float64
sd_occupancy        76057 non-null float64
days_observed       76057 non-null int64
state_pm            76057 non-null float64
abs_pm              76057 non-null float64
latitude            76057 non-null float64
longitude           76057 non-null f

In [315]:
pems_conflation_result_df["source"].fillna("nearest", inplace = True)

In [176]:
pems_conflation_result_df= pems_conflation_result_df.to_crs(epsg = 4326)

In [178]:
pems_conflation_result_df.to_file(data_interim_dir + "mtc/pems_conflation_result_new.geojson",
                                  driver = "GeoJSON")

In [179]:
pems_conflation_result_df.head(3)

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,abs_pm,latitude,longitude,year,geometry,shstReferenceId,roadway,tomtom_shieldnum,tomtom_rtedir,snap_distance
0,400000,4,101,S,ML,AM,3,16383.0,16237.843137,691.559263,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488
1,400000,4,101,S,ML,EA,3,4366.5,4376.357143,102.251993,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488
2,400000,4,101,S,ML,EV,3,8523.0,8434.255814,420.135754,...,459.52,38.081167,-122.547606,2005,POINT (-122.547606 38.081167),df209cac09d05a2f7e0a22dc8e931df8,motorway,101,S,3.17488


In [61]:
pems_df.station.nunique()

3802

In [62]:
3332 + 393

3725

In [64]:
pems_gdf.station.nunique()

3700

In [65]:
pems_conflation_result_df.station.nunique()

3700

In [69]:
pems_conflation_result_df.groupby(["shstReferenceId", "station", "year"]).count()

district  route  direction  \
shstReferenceId                  station year                               
000619909cfedfdc3ae846759247e09f 404433  2014         5      5          5   
                                 407341  2016         5      5          5   
                                         2017         5      5          5   
001faa1f8bf0fafb4298b7438a83b506 400615  2005         5      5          5   
                                         2006         5      5          5   
                                         2007         5      5          5   
                                         2008         5      5          5   
                                         2009         5      5          5   
                                         2010         5      5          5   
                                         2011         5      5          5   
                                         2013         5      5          5   
                                         2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017        10     10         10   
                                 414478  2017         5      5          5   
                                 414479  2017         5      5          5   
                                 414480  2017         5      5          5   
                                 419158  2017         1      1          1   
                                 419159  2017         2      2          2   
00441ada4f77ea0e20dd17acd688e289 404649  2014         5      5          5   
                                         2015         5      5          5   
005586c0c986e59d4dcd979513f7e8e6 401943  2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 402067  2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
...                                                 ...    ...        ...   
ff90d3fb5034f779cc6c4841f2fbcfc5 402096  2017         5      5          5   
                                 408152  2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 408153  2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 408157  2016         5      5          5   
                                         2017         5      5          5   
ff9ba0a399cb6a07872339ed82edea56 400703  2016         5      5          5   
                                         2017         5      5          5   
                                 403325  2013         5      5          5   
                                         2014         5      5          5   
                                         2015         5      5          5   
                                         2016         5      5          5   
                                         2017         5      5          5   
                                 412871  2016         5      5          5   
                                         2017         5      5          5   
ff9fb0954535136f65fa9fda0774b468 404877  2013         5      5          5   
                                         20